## Preparando o dataframe referente aos bairros, regionais renda e população

In [1]:
# Fontes extraídas do link passado pelo professor Luiz Celso
# https://gitlab.com/luizcelso/mobilidade-curitiba-covid19/-/raw/master/data/bairros_por_regional.csv
# https://gitlab.com/luizcelso/mobilidade-curitiba-covid19/-/raw/master/data/renda.csv
import pandas as pd


In [2]:
# Importando dataframes com os dados economicos dos bairros e uma lista de bairros e suas respectivas regionais

df_regionais = pd.read_csv("../dados/bairro_regional.csv", sep=',')
df_regionais['Bairro'] = df_regionais['Bairro'].str.upper()
df_renda = pd.read_csv("../dados/renda_bairro.csv", sep=',')

df_renda.head(3)

,Bairros,código,Média de Media latitude,Média de media longitude,Tempo,RMR,Resíduo padrão,Label,Área (km²),Homens,Mulheres,População Total,Domicilios particulares,Rendimento mensal médio por responsáveis dos domicílios (R$)
0,ALTO BOQUEIRAO,47,"-25,5333703446815","-49,2421297196611",19,"2,9","-2,44201144966097",Alto Boqueirão,"12,11",24824,26331,51155,14301,"1110,63"
1,LAMENHA PEQUENA,63,"-25,3663199254971","-49,329475340023",12,"2,4","-1,94536383937777",Lamenha Pequena,"3,4",353,348,701,188,"665,99"
2,XAXIM,23,"-25,5090296034033","-49,267007631085",13,"3,6","-1,46226860276454",Xaxim,"8,92",26747,27944,54691,15482,"1211,6"


In [3]:
# Filtrando e tratando os dados do dataframe

renda = df_renda[['Bairros', 'Label','População Total', 'Rendimento mensal médio por responsáveis dos domicílios (R$)']]
renda = renda.rename(columns={'Rendimento mensal médio por responsáveis dos domicílios (R$)': 'Renda'})
renda['Renda'] = renda['Renda'].replace(',','.', regex=True).astype(float)
renda['Renda Bruta'] = renda['População Total'] * renda['Renda']
renda.head(3)

,Bairros,Label,População Total,Renda,Renda Bruta
0,ALTO BOQUEIRAO,Alto Boqueirão,51155,1110.63,56814277.65
1,LAMENHA PEQUENA,Lamenha Pequena,701,665.99,466858.99
2,XAXIM,Xaxim,54691,1211.60,66263615.60


In [4]:
df_merge = renda.merge(df_regionais, how='left', left_on='Bairros', right_on='Bairro', suffixes=(False, False))
df_merge[df_merge.isna().any(axis=1)]

,Bairros,Label,População Total,Renda,Renda Bruta,Bairro,Regional


In [7]:
df_merge.head(3)

,Bairros,Label,População Total,Renda,Renda Bruta,Bairro,Regional
0,ALTO BOQUEIRAO,Alto Boqueirão,51155,1110.63,56814277.65,ALTO BOQUEIRAO,Boqueirão
1,LAMENHA PEQUENA,Lamenha Pequena,701,665.99,466858.99,LAMENHA PEQUENA,Santa Felicidade
2,XAXIM,Xaxim,54691,1211.60,66263615.60,XAXIM,Boqueirão


In [5]:
df_merge[df_merge['Regional'] == 'CIC']

,Bairros,Label,População Total,Renda,Renda Bruta,Bairro,Regional
3,RIVIERA,Riviera,203,423.13,8.589539e+04,RIVIERA,CIC
4,SAO MIGUEL,São Miguel,4911,409.43,2.010711e+06,SAO MIGUEL,CIC
10,AUGUSTA,Augusta,3617,619.54,2.240876e+06,AUGUSTA,CIC
14,CIDADE INDUSTRIAL DE CURITIBA,Cidade Industrial,157461,639.09,1.006318e+08,CIDADE INDUSTRIAL DE CURITIBA,CIC


In [7]:
# Agrupando por regionais e somando a renda bruta e população. Depois, criando a renda média por regional

regionais = df_merge.groupby('Regional').agg({'População Total':'sum','Renda Bruta':'sum'})
regionais['Renda Média'] =  regionais['Renda Bruta']/regionais['População Total']
regionais.sort_values('Renda Média')

,População Total,Renda Bruta,Renda Média
Regional,,,
Tatuquara,46149,2.224219e+07,481.964742
CIC,166192,1.049692e+08,631.614234
Bairro Novo,124330,1.146315e+08,921.994247
Pinheirinho,121491,1.141769e+08,939.797274
Cajuru,202787,2.246884e+08,1108.001763
Boqueirão,188192,2.422735e+08,1287.374094
Santa Felicidade,100762,1.510019e+08,1498.599244
Boa Vista,218741,3.321569e+08,1518.493802
Fazendinha,194675,3.863401e+08,1984.538745


In [10]:
regionais.to_csv("../dados/dados_limpos/renda_media_regionais.csv")